### 使用Apriori算法来发现频繁项

Apriori中的辅助函数

In [1]:
from numpy import *

def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))#use frozen set so we
                            #can use it as a key in a dict    

def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if can not in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [3]:
dataSet = loadDataSet()

In [16]:
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [9]:
a = [[1],[2]]
list(map(frozenset,a))

[frozenset({1}), frozenset({2})]

In [14]:
x = {1}
x.issubset([1,2,3])

True

测试：

In [19]:
D = list(map(set, dataSet))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [20]:
L1, suppData0 = scanD(D, C1, 0.5)

In [21]:
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [22]:
suppData0

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75}

Apriori算法

In [23]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [24]:
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [32]:
aprioriGen(L1, 2)

[frozenset({2, 5}),
 frozenset({3, 5}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({1, 3})]

In [43]:
L, suppData = apriori(dataSet)

In [44]:
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [45]:
suppData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({1, 5}): 0.25,
 frozenset({1, 2}): 0.25,
 frozenset({2, 3, 5}): 0.5}

In [38]:
L, suppData = apriori(dataSet, minSupport=0.7)

In [39]:
L

[[frozenset({5}), frozenset({2}), frozenset({3})], [frozenset({2, 5})], []]

In [40]:
suppData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5}

### 从频繁项集中挖掘关联规则

In [42]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [50]:
for i in range(1, len(L)):
    for freqSet in L[i]:
        H1 = [frozenset([item]) for item in freqSet]
        print(H1)

[frozenset({2}), frozenset({3})]
[frozenset({3}), frozenset({5})]
[frozenset({2}), frozenset({5})]
[frozenset({1}), frozenset({3})]
[frozenset({2}), frozenset({3}), frozenset({5})]


In [51]:
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [53]:
L[2]

[frozenset({2, 3, 5})]

In [60]:
H1

[frozenset({2}), frozenset({3}), frozenset({5})]

In [62]:
len(freqSet)

3

In [75]:
L, suppData = apriori(dataSet, minSupport=0.5)

In [76]:
rules = generateRules(L, suppData, minConf=0.7)
rules

frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({1}) --> frozenset({3}) conf: 1.0


[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]

In [77]:
rules = generateRules(L, suppData, minConf=0.5)
rules

frozenset({3}) --> frozenset({2}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({5}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({3}) --> frozenset({1}) conf: 0.6666666666666666
frozenset({1}) --> frozenset({3}) conf: 1.0
frozenset({5}) --> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf: 0.6666666666666666


[(frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]

In [80]:
aprioriGen(H1, 2)

[frozenset({2, 3}), frozenset({2, 5}), frozenset({3, 5})]

### 示例：发现国会投票中的模式

### 示例：发现毒蘑菇的相似特征

In [81]:
mushDatSet = [line.split() for line in open("./mushroom.dat").readlines()]

In [83]:
len(mushDatSet)

8124

In [84]:
L, suppData = apriori(mushDatSet, minSupport=0.3)

In [86]:
for item in L[1]:
    if item.intersection('2'):
        print(item)

frozenset({'28', '2'})
frozenset({'53', '2'})
frozenset({'23', '2'})
frozenset({'34', '2'})
frozenset({'36', '2'})
frozenset({'59', '2'})
frozenset({'63', '2'})
frozenset({'67', '2'})
frozenset({'76', '2'})
frozenset({'85', '2'})
frozenset({'86', '2'})
frozenset({'90', '2'})
frozenset({'93', '2'})
frozenset({'39', '2'})


In [88]:
for item in L[2]:
    if item.intersection('2'):
        print(item)

frozenset({'39', '53', '2'})
frozenset({'90', '53', '2'})
frozenset({'2', '53', '86'})
frozenset({'85', '53', '2'})
frozenset({'34', '53', '2'})
frozenset({'39', '28', '2'})
frozenset({'28', '53', '2'})
frozenset({'90', '28', '2'})
frozenset({'28', '2', '86'})
frozenset({'28', '85', '2'})
frozenset({'63', '28', '2'})
frozenset({'59', '28', '2'})
frozenset({'28', '34', '2'})
frozenset({'39', '93', '2'})
frozenset({'39', '2', '86'})
frozenset({'39', '85', '2'})
frozenset({'39', '76', '2'})
frozenset({'39', '67', '2'})
frozenset({'39', '2', '36'})
frozenset({'39', '34', '2'})
frozenset({'39', '2', '23'})
frozenset({'86', '93', '2'})
frozenset({'85', '93', '2'})
frozenset({'93', '36', '2'})
frozenset({'23', '93', '2'})
frozenset({'90', '93', '2'})
frozenset({'90', '86', '2'})
frozenset({'90', '85', '2'})
frozenset({'90', '63', '2'})
frozenset({'90', '59', '2'})
frozenset({'90', '36', '2'})
frozenset({'90', '34', '2'})
frozenset({'90', '23', '2'})
frozenset({'90', '39', '2'})
frozenset({'86